### Import numpy , pandas and OS Library, default code snippet given by Kaggle

In [1]:
import numpy as np 
import pandas as pd 

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/gemma/keras/gemma_2b_en/1/config.json
/kaggle/input/gemma/keras/gemma_2b_en/1/tokenizer.json
/kaggle/input/gemma/keras/gemma_2b_en/1/metadata.json
/kaggle/input/gemma/keras/gemma_2b_en/1/model.weights.h5
/kaggle/input/gemma/keras/gemma_2b_en/1/assets/tokenizer/vocabulary.spm
/kaggle/input/gemma/pytorch/2b/1/config.json
/kaggle/input/gemma/pytorch/2b/1/gemma-2b.ckpt
/kaggle/input/gemma/pytorch/2b/1/tokenizer.model
/kaggle/input/python-codes-data-25k/python_codes_25k_train_data.json
/kaggle/input/data-assistants-with-gemma/submission_categories.txt
/kaggle/input/data-assistants-with-gemma/submission_instructions.txt
/kaggle/input/cleaned-python-25k-data/extracted_data.json
/kaggle/input/parquetfile-python-25k/0000.parquet


### Install Keras-NLP and Keras

In [2]:
!pip install -q -U keras-nlp
!pip install -q -U keras>=3

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.0.5 which is incompatible.
tensorflowjs 4.16.0 requires packaging~=23.1, but you have packaging 21.3 which is incompatible.


### One can also try calling pretrained models from Hf, but will need latest transformers library for tokeniser

In [3]:
# !pip install transformers==4.38.0
# from transformers import AutoTokenizer, AutoModelForCausalLM
# tokenizer = AutoTokenizer.from_pretrained("google/gemma-2b-it")
# model = AutoModelForCausalLM.from_pretrained("google/gemma-2b-it")

#Another notebook another day

### Go to hf and search for flytech/python-codes-25k and download parquet file and upload the dataset on kaggle and call it by pandas

In [4]:
import pandas as pd
file = pd.read_parquet("/kaggle/input/parquetfile-python-25k/0000.parquet")

### We will pass parquet file by extracting Instruction and output column and will convert it into List + to save memory will delete the file variable later, also will only take rows from 0 to 999

In [5]:
data=file.apply(lambda row:f"Instruction:\n{row.instruction}\n\nResponse:\n{row.output}",axis=1).values.tolist()[:1000]

In [6]:
del file

### You can see by print statement what our "data" variable contains

In [7]:
print(data[1])

Instruction:
Create a shopping list based on my inputs!

Response:
```python
shopping_list = {}
while True:
    item = input('Enter an item or type 'done' to finish: ')
    if item == 'done': break
    quantity = input(f'Enter the quantity for {item}: ')
    shopping_list[item] = quantity
print(f'Your shopping list: {shopping_list}')
```


In [8]:
import keras
import keras_nlp

2024-02-21 18:23:49.165705: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-21 18:23:49.165806: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-21 18:23:49.288029: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


### Import necessary backend and to avoid memory fragmentation use the below code

In [9]:
import os

os.environ["KERAS_BACKEND"] = "jax" #or torch or tensorflow
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"]="1.00"

### Call the Gemma model by adding model from right hand side by add model button and add gemma 2billion 94.3 GB Model

In [10]:
gemma_lm = keras_nlp.models.GemmaCausalLM.from_preset("gemma_2b_en")

Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/1' to your Kaggle notebook...
Attaching 'config.json' from model 'keras/gemma/keras/gemma_2b_en/1' to your Kaggle notebook...
Attaching 'model.weights.h5' from model 'keras/gemma/keras/gemma_2b_en/1' to your Kaggle notebook...
Attaching 'tokenizer.json' from model 'keras/gemma/keras/gemma_2b_en/1' to your Kaggle notebook...
Attaching 'assets/tokenizer/vocabulary.spm' from model 'keras/gemma/keras/gemma_2b_en/1' to your Kaggle notebook...
normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.


In [11]:
gemma_lm.summary()

Preprocessor: "gemma_causal_lm_preprocessor"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Tokenizer (type)                                   ┃                                             Vocab # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ gemma_tokenizer (GemmaTokenizer)                   │                                             256,000 │
└────────────────────────────────────────────────────┴─────────────────────────────────────────────────────┘

Model: "gemma_causal_lm"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ padding_mask (InputLayer)     │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_ids (InputLayer)        │ (None, None)              │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ gemma_backbone                │ (None, None, 2048)        │   2,506,172,416 │ padding_mask[0][0],        │
│ (GemmaBackbone)               │                           │                 │ token_ids[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ token_embedding               │ (None, None, 256000)      │     524,288,000 │ gemma_backbone[0][0]       │
│ (ReversibleEmbedding)         │                           │                 │                            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 2,506,172,416 (9.34 GB)

 Trainable params: 2,506,172,416 (9.34 GB)

 Non-trainable params: 0 (0.00 B)

### Use Lora to finetune

In [12]:
gemma_lm.backbone.enable_lora(rank=4)

### To save memory decrease the context window

In [13]:
# Limit the input sequence length to 512 (to control memory usage).
gemma_lm.preprocessor.sequence_length = 128
# Use AdamW (a common optimizer for transformer models).
optimizer = keras.optimizers.AdamW(
    learning_rate=5e-6,
    weight_decay=0.01,
)
# Exclude layernorm and bias terms from decay.
optimizer.exclude_from_weight_decay(var_names=["bias", "scale"])

gemma_lm.compile(
    loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=optimizer,
    weighted_metrics=[keras.metrics.SparseCategoricalAccuracy()],
)
gemma_lm.fit(data, epochs=1, batch_size=1)

I0000 00:00:1708539946.840267      86 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
W0000 00:00:1708539946.917229      86 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


1000/1000 ━━━━━━━━━━━━━━━━━━━━ 247s 206ms/step - loss: 1.0880 - sparse_categorical_accuracy: 0.6695


### Save the model, such that after inferencing, you can call the finetuned model and again finetune on more number of rows

In [14]:
gemma_lm.save("version_finetuned.keras")

### You can see it answers the question brilliantly

In [15]:
instruction="write a code for creating a list in python"
response=""
prompt = f"Instruction:\n{instruction}\n\nResponse:\n{response}"
print(gemma_lm.generate(prompt, max_length=128))

W0000 00:00:1708540280.859736      26 graph_launch.cc:671] Fallback to op-by-op mode because memset node breaks graph update


Instruction:
write a code for creating a list in python

Response:
# Create a list of strings.
list_of_strings = [‘apple’, ‘banana’, ‘cherry’, ‘orange’,]
print(list_of_strings)

Explanation:
In this code, first we create a list of strings.
Then we print the list of strings.

